<img src="https://blastim.ru/wp-content/uploads/wpjobboard/company/49/company-logo/bostongene-logo.jpg" width=500 align="center">

# Практикум

## Загрузка прочтений

На лекции мы с вами говорили о базе данных NCBI SRA. Для того, чтобы загружать данные оттуда, необходим специальный пакет SRA Toolkit, а также программа <code>fastq-dump</code>. По неведомым мне причинам fastq-dump плохо ставится на Google Colab, поэтому мы загрузим данные из другой большой базы &mdash; <a href="https://www.ebi.ac.uk/arrayexpress/" target="_blank">ArrayExpress</a>.

Из ArrayExpress <a href="https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-9724/" target="_blank">E-MTAB-9724</a> загрузим несколько прочтений экспериментов RNA-Seq клеточных линий &mdash; с нокаутом гена <i>STK11</i> и с диким типом. <b>Загрузка прочтений может занять много времени!</b>

In [ ]:
!mkdir raw_reads
!cd raw_reads && mkdir sample_8 sample_9 sample_10 sample_11
!cd raw_reads/sample_8 && wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/003/ERR4781423/ERR4781423_1.fastq.gz && wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/003/ERR4781423/ERR4781423_2.fastq.gz
!cd raw_reads/sample_9 && wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/004/ERR4781424/ERR4781424_1.fastq.gz && wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/004/ERR4781424/ERR4781424_2.fastq.gz
!cd raw_reads/sample_10 && wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/005/ERR4781425/ERR4781425_1.fastq.gz && wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/005/ERR4781425/ERR4781425_2.fastq.gz
!cd raw_reads/sample_11 && wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/006/ERR4781426/ERR4781426_1.fastq.gz && wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/006/ERR4781426/ERR4781426_2.fastq.gz

Теперь установим программу FastQC и проведём контроль качества прочтений.

In [ ]:
!sudo apt install fastqc
!fastqc raw_reads/sample_8/*.fastq.gz
!fastqc raw_reads/sample_9/*.fastq.gz
!fastqc raw_reads/sample_10/*.fastq.gz
!fastqc raw_reads/sample_11/*.fastq.gz

Теперь можно загрузить .html-отчёт о качестве себе на компьютер и посмотреть, всё ли хорошо с прочтениями.

## Установка kallisto и подсчёт экспрессии

Теперь мы загружаем и устанавливаем саму программу <code>kallisto</code>. При помощи неё мы сможем провести оценку количества прочтений, ложащихся на разные транскрипты.

In [ ]:
!git clone https://github.com/pachterlab/kallisto.git
!apt-get install autoconf
!cd kallisto && mkdir build && cd build && cmake .. && make

Теперь загружаем и распаковываем файл с индексированным транскриптомом. В общем случае можно сгенерировать его самому, однако в случае с человеком он обычно уже есть в открытом доступе. Если потребуется делать индекс самостоятельно, то следует воспользоваться командой <code>kallisto index</code>.

In [ ]:
!wget https://github.com/pachterlab/kallisto-transcriptome-indices/releases/download/ensembl-96/homo_sapiens.tar.gz
!tar -xvzf homo_sapiens.tar.gz && rm homo_sapiens.tar.gz

Итак, всё готово к запуску. Итоговые экспрессии выведем в папку <code>expression_data</code>. Для начала запустим на одном образце и посмотрим, что у нас получилось.

In [ ]:
!mkdir expression_data
!./kallisto/build/src/kallisto quant -i homo_sapiens/transcriptome.idx -o expression_data/sample_8 raw_reads/sample_8/ERR4781423_1.fastq.gz raw_reads/sample_8/ERR4781423_2.fastq.gz

Посмотрим, какой выход дала нам программа.

In [ ]:
!ls expression_data/sample_8

Теперь посмотрим, что хранится в файле <code>abundance.tsv</code>.

In [ ]:
!head expression_data/sample_8/abundance.tsv

Отлично! Теперь мы видим, что напротив каждого <code>target_id</code> в конце строчки стоит колонка <code>tpm</code>. Повторим предыдущую команду для всех прочтений.

In [ ]:
!./kallisto/build/src/kallisto quant -i homo_sapiens/transcriptome.idx -o expression_data/sample_9 raw_reads/sample_9/ERR4781424_1.fastq.gz raw_reads/sample_9/ERR4781424_2.fastq.gz
!./kallisto/build/src/kallisto quant -i homo_sapiens/transcriptome.idx -o expression_data/sample_10 raw_reads/sample_10/ERR4781425_1.fastq.gz raw_reads/sample_10/ERR4781425_2.fastq.gz
!./kallisto/build/src/kallisto quant -i homo_sapiens/transcriptome.idx -o expression_data/sample_11 raw_reads/sample_11/ERR4781426_1.fastq.gz raw_reads/sample_11/ERR4781426_2.fastq.gz


[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 188,753
[index] number of k-mers: 109,544,288
tcmalloc: large alloc 3221225472 bytes == 0x561dc2c82000 @  0x7fc1e332f887 0x561dc1611ae2 0x561dc160a071 0x561dc15df05a 0x7fc1e220abf7 0x561dc15e30ea
[index] number of equivalence classes: 760,757
[quant] running in paired-end mode
[quant] will process pair 1: raw_reads/sample_9/ERR4781424_1.fastq.gz
                             raw_reads/sample_9/ERR4781424_2.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 25,354,517 reads, 22,595,696 reads pseudoaligned
[quant] estimated average fragment length: 230.865
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 1,114 rounds


[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 188,753
[index] number of k-mers: 109,544,288
tcmal

## Создание итоговой таблицы экспрессий

Теперь склеим все обсчитанные экспрессии в одну сводную таблицу.

In [ ]:
expressions_tpm = pd.concat(
    [pd.read_csv(
        "expression_data/" + expression + "/abundance.tsv",
        sep="\t", index_col=0, header=0, usecols=["target_id", expression],
        names=["target_id", "1", "2", "3", expression]
    ) for expression in ["sample_8", "sample_9", "sample_10", "sample_11"]],
    axis=1
)
expressions_tpm.columns = ["sample_8", "sample_9", "sample_10", "sample_11"]
expressions_tpm.head()

## Анализ экспрессий

Теперь нас ожидает самая интересная задача &mdash; поиск дифференциально экспрессированных генов.

### rpy2

Из-за того, что анализ дифференциальной экспрессии в основном написан для R, мы будем пользоваться пакетом rpy2, чтобы выполнять код R в Python.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import rpy2
from rpy2.robjects import r, pandas2ri

In [ ]:
%load_ext rpy2.ipython
pandas2ri.activate()

Вот таким образом выглядит ячейка, которая будет выполняться как R-скрипт:

In [ ]:
%%R
a <- c(1,2,3)
a <- a**2
print(a)

Для того, чтобы объекты из окружения R попали в окружение Python (и наоборот), необходимо выполнить следующее (обратите внимание, что выгружаемый объект имеет непривычный для Python тип):

In [ ]:
a = 20
%R -i a
%R a <- a**2
%R -o a
print(a, type(a))

Если хочется более подробно овладеть rpy2, то лучше прочитайте <a href="https://rpy2.github.io/doc/latest/html/index.html" target="_blank">его документацию</a>.

### Анализ дифференциальной экспрессии при помощи DESeq2

Сейчас необходимо будет поставить пакеты для R в наше окружение. Следующие строки будут начинаться с <code>warnings.warn(x, RRuntimeWarning)</code>, однако не стоит переживать. Это лишь говорит о том, что эта строчка должна быть окрашена в красный цвет. После успешной установки пакетов появится поле для ввода: можно будет завершить эту ячейку, нажав на знак <b>стоп</b> (закрашенный квадратик) справа.

In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")

library("DESeq2")
library(ggplot2)

In [ ]:
%%R
library("DESeq2")
library(ggplot2)

На этом этапе мы перегружаем наш датафрейм из Python в R, чтобы дальше с ним работать там. Чтобы убедиться, что у нас нет проблем с типами данных, чётко зададим значения матрицы каунтов как <code>int32</code>.

In [ ]:
expr_matrix = expressions_count.copy()
expr_matrix = expr_matrix.astype("int32")
expr_matrix.index.name = ""
gene_id = expr_matrix.index.copy()
%R -i expr_matrix
%R -i gene_id

In [ ]:
%%R
rownames(expr_matrix) = make.names(gene_id, unique=TRUE)
head(expr_matrix)

Теперь необходимо создать аннотацию для каждого семпла. Аннотация представляет из себя ещё один датафрейм. Также мы сделаем новую переменную <code>dds</code> специального типа, который используется при работе в <code>DESeq2</code>. В нашем простейшем случае у нас, помимо названий колонок, будут условия, которыми они разделяются (WT против Mut). Считать GLM мы будем, указывая в качестве предиктора именно их. Важно иметь в виду, что модель может оценивать дифференциальную экспрессию в зависимости от сразу нескольких факторов, которые могут быть в том числе и количественные.

In [ ]:
%%R
condition <- c("WT", "WT", "Mut", "Mut")
name <- c("sample_8", "sample_9", "sample_10", "sample_11")
colData <- data.frame(condition, name)
dds <- DESeqDataSetFromMatrix(expr_matrix, colData, design=~condition)
print(dds)

Теперь мы получили специальный объект класса <code>DESeqDataSet</code>, с которым мы выполним функцию <code>DESeq</code>, которая выполняет практически весь анализ дифференциальной экспрессии, который только вшит в пакет.

In [ ]:
%%R
dds <- DESeq(dds)

Теперь посмотрим, что поменялось в переменной <code>dds</code>.

In [ ]:
%%R
print(dds)

Посмотрим, какие же из транскриптов дифференциально экспрессированы между двумя обозначенными нами группами. Сначала выведем статистику по каждому транскрипту.

In [ ]:
%%R
res <- results(dds)
head(results(dds))

Упорядочим их по p-value.

In [ ]:
%%R
res <- res[order(res$padj),]
head(res)

Теперь можно визуализировать разницу между самыми различающимися транскриптами.

In [ ]:
%%R -w 1000
par(mfrow=c(2,3))

plotCounts(dds, gene="", intgroup="condition")

Визуализировать результаты удобно при помощи volcano plot.

In [ ]:
%%R -w 1000
par(mfrow=c(1,1))

with(res, plot(log2FoldChange, -log10(pvalue), pch=20, main="Volcano plot", xlim=c(-3,3)))

#Покрасим синим точки, у которых p_adj < 0.01, а красным точки, у которых p_adj < 0.01 и logFC > 2.
with(subset(res, padj<.01 ), points(log2FoldChange, -log10(pvalue), pch=20, col="blue"))
with(subset(res, padj<.01 & abs(log2FoldChange)>2), points(log2FoldChange, -log10(pvalue), pch=20, col="red"))

# Домашнее задание

## Задание 1

До этого мы с вами работали на уровне транскриптов. Однако транскрипты сами по себе мало о чём нам говорят &mdash; нам интересно посмотреть, какие гены дифференциально экспрессированы. Для этого перед тем, как анализировать дифференциальную экспрессию, преобразуйте таблицу с экспрессиями &mdash; сложите каунты транскриптов, относящихмя к одному гену, и создайте новую таблицу, в которой будут суммы транскриптов по генам. Вместо Ensembl ID генов (<code>ENSGXXXX</code>) используйте названия (<code>gene symbol</code>).

<i>Подсказка: соотнесение транскриптов к генам можно найти на сайте Ensembl. Также вы можете для ваших целей использовать пакет </i><code>myGene</code><i> на Python 3.</i>

## Задание 2

Загрузите прочтения ещё четырёх образцов из исследования &mdash; две линии с нокаутом <i>KEAP1</i>, а также две линии с наукаутом по обоим генам. Выполните обработку, аналогичную той, которую мы делали на занятии: посмотрите на дифференциально экспрессированные гены для (1) просто нокаутов по <i>KEAP1</i> и для (2) нокаутов по обоим генам.

Являются ли дифференциально экспрессированные гены во втором случае объединением дифференциально экспрессированных генов по нокаутом <i>KEAP1</i> и <i>STK11</i>? Почему?

## Задание 3

Постройте PCA для получившихся образцов. Как именно это делается можно прочитать <a href="https://www.machinelearningmastery.ru/pca-using-python-scikit-learn-e653f8989e60/" target="_blank">тут</a> (в R это делается вообще одной функцией <code>prcomp</code>). Объясните получившиеся результаты. Что именно вы ожидали увидеть на этом графике PCA?